# TFRecord

- Train/Test/Validation Dataset을 하나의 파일로 디스크(HDD)에 저장하는 Tensorflow 파일저장형식.
    - Train 시 파일로 저장된 Raw 데이터를 모델에 입력할 때 **데이터 입력, Label parsing**하는 것이 학습 속도를 떨어트리는 원인이 된다.
        - batch size 만큼의 이미지 파일을 읽어 들이면 batch size만큼의 파일 입력작업이 발생한다.
        - annotation 파일에서 정답(y)값을 parsing 하는 것도 속도 저하의 원인이 된다.
    - Tensorflow에서 학습시 데이터셋을 읽어들이는 속도를 향상시키기 위해 데이터들에 Serialization을 수행해서 하나의 파일로 저장할 수 있는 TFRecord 파일 포맷을 제공한다.
- https://www.tensorflow.org/tutorials/load_data/tfrecord   
> - **직렬화(Serialization):** 메모리에 저장된 다양한 타입의 값을 디스크(네트워크)에 저장할 수 있는 상태로 변환하는 것.
> - **binary data:** 디스크에 저장되는 0, 1로 구성된 데이터

- tf.train.Example
    - 하나의 데이터 포인트를 TFRecord에 저장하기 위해 변환하는 타입. 하나의 데이터포인트를 tf.train.Example 의 객체로 변환해서 직렬화 한 뒤 저장한다.

- tf.train.Feature
    - 하나의 데이터를 구성하는 속성(feature)들을 변환하는 클래스.
    - tf.train.Feature는 다음 세가지 타입을 지원한다.
        - tf.train.BytesList – string, bytes 타입을 변환
        - tf.train.FloatList –  float(float32), double(float64) 타입을 변환
        - tf.train.Int64List –  int, uint, bool, enum 타입을 변환

- tf.train.Example의 형태
```python
{
    "feature명":tf.train.Feature타입객체,
    "feature명":tf.train.Feature타입객체,
    ...
}
```


In [1]:
import tensorflow as tf
import numpy as np

In [2]:
# The following functions can be used to convert a value to a type compatible
# with tf.train.Example.

# 문자열(str), bytes(일반파일-ex. 이미지파일, 음성파일......)를 변환하는 함수
def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

# 실수형 값을 변환하는 함수
def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

# 정수계열값(int, uint, bool, enum)을 반환하는 함수
def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [3]:
# 정수
v = _int64_feature(100)
print(type(v))
v

<class 'tensorflow.core.example.feature_pb2.Feature'>


int64_list {
  value: 100
}

In [4]:
# bool
# v = _int64_feature(True)
v = _int64_feature(False)
v

int64_list {
  value: 0
}

In [5]:
# float
v = _float_feature(0.234)
v

float_list {
  value: 0.23399999737739563
}

In [6]:
# 문자열 -> bytes타입으로 변환 후 변환
v = _bytes_feature(b'abc')
v

bytes_list {
  value: "abc"
}

In [7]:
s = '홍길동 abc'
# 문자열 -> bytes로 encoding
s2 = s.encode('utf-8')
v = _bytes_feature(s2)
v

bytes_list {
  value: "\355\231\215\352\270\270\353\217\231 abc"
}

In [13]:
print(s2)
print(s2.decode('utf-8')) # bytes -> 문자열로 변환

b'\xed\x99\x8d\xea\xb8\xb8\xeb\x8f\x99 abc'


'홍길동 abc'

In [18]:
# 파일
with open('labelme_data/cat.jpg', 'rb') as fr:
    img = fr.read()
print(type(img))
v = _bytes_feature(img)
print(type(v))

<class 'bytes'>
<class 'tensorflow.core.example.feature_pb2.Feature'>


## Example 생성 예제
- 문자열을 key로 Feature를 값으로 가지는 dictionary를 이용해 생성

In [19]:
name = '홍길동'
age = 32
tall = 182.5
picture = img
label = 0
# name, age, tall, picture: X + label: y -> Data point (1개의 정보)

In [23]:
feature_dict = {
    # name: 문자열, value: Feature
    "name": _bytes_feature(name.encode('utf-8')),
    "age": _int64_feature(age),
    "tall": _float_feature(tall),
    "picture": _bytes_feature(img),
    "label": _int64_feature(label)
}

example = tf.train.Example(features=tf.train.Features(feature=feature_dict))

## Feature 직렬화 예제
- Feature객체/Example객체.SerializeToString()
    - 각 Featuer(Feature객체를 통해 호출), Example의 Feature들(Example객체를 통해 호출)을 파일로 출력할 수 있도록 직렬화(bytes로 변환) 한다.
        - Feature객체->bytes 로 변환
    - tfrecord 파일로 출력하기 전에 Example의 Feature들은 직렬화 되어야 한다.

In [26]:
type(example)
# Example 객체 -----> 디스크에 저장하기해서 bytes로 변환 -----> 직렬화가능 형태로 변환
# Example 객체 --> bytes
se = example.SerializeToString()
se # tfrecord write를 이용해서 출력

b'\n\xe6\xf7!\n\x0e\n\x05label\x12\x05\x1a\x03\n\x01\x00\n\x0c\n\x03age\x12\x05\x1a\x03\n\x01 \n\x9b\xf7!\n\x07picture\x12\x8e\xf7!\n\x8a\xf7!\n\x86\xf7!\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xe1\x00HExif\x00\x00II*\x00\x08\x00\x00\x00\x01\x00\x98\x82\x02\x00&\x00\x00\x00\x1a\x00\x00\x00\x00\x00\x00\x00This content is subject to copyright.\x00\xff\xe1\x15\xc6http://ns.adobe.com/xap/1.0/\x00<?xpacket begin="\xef\xbb\xbf" id="W5M0MpCehiHzreSzNTczkc9d"?> <x:xmpmeta xmlns:x="adobe:ns:meta/" x:xmptk="Adobe XMP Core 5.6-c145 79.163499, 2018/08/13-16:40:22        "> <rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"> <rdf:Description rdf:about="" xmlns:Iptc4xmpCore="http://iptc.org/std/Iptc4xmpCore/1.0/xmlns/" xmlns:plus="http://ns.useplus.org/ldf/xmp/1.0/" xmlns:photoshop="http://ns.adobe.com/photoshop/1.0/" xmlns:GettyImagesGIFT="http://xmp.gettyimages.com/gift/1.0/" xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:xmp="http://ns.adobe.com/xa

# TFRecord 생성 예제

In [27]:
# TFRecord에 직렬화할 대상 데이터들, (1000, 4)
N_DATA = 1000 # 총 데이터개수
feature0 = np.random.choice([True, False], N_DATA) # bool 타입 feature
feature1 = np.random.randint(0, 5, N_DATA) # 정수 feature
s = np.array([b'cat', b'dog', b'chicken', b'horse', b'goat'])
feature2 = s[feature1] # 1000개의 문자열 bytes(문자열) feature
feature3 = np.random.randn(N_DATA) #float feature

In [31]:
num = 0
for f0, f1, f2, f3 in zip(feature0, feature1, feature2, feature3):
    print(f0, f1, f2, f3, sep=', ')
    num+=1
    if num == 5:
        break

False, 4, b'goat', -0.696893709619397
False, 0, b'cat', -0.6080984249954645
True, 0, b'cat', 0.8929877930951521
True, 3, b'horse', -1.4121969285196798
False, 4, b'goat', -0.9274218143047124


## tf.train.Example 생성 및 직렬화(Serialize)
1. 각 관측값의 Feature들 하나하나는 위의 함수 중 하나를 사용하여 tf.train.Feature 타입으로 변환(인코딩)한다.
2. 1번에서 변환된 Feature들에 이름(key)를 붙여 tf.train.Example객체를 생성한다.
3. Example의 SerializeToString() 메소드를 이용해 직렬화 한다. 

In [34]:
def serialize_example(f0, f1, f2, f3):
    """
    1개의 Datapoint를 구성하는 속성값들(f0, f1, f2, f3)을 받아서 
    Example을 생성한 뒤 Serialize(bytes)시켜 return하는 함수
    f0: bool
    f1: int
    f2: bytes(str)
    f3: float
    """
    feature_dict = {
        'f0':_int64_feature(f0),
        'f1':_int64_feature(f1),
        'f2':_bytes_feature(f2),
        'f3':_float_feature(f3)
    }
    # Example 객체 생성
    example = tf.train.Example(features=tf.train.Features(feature=feature_dict))
    # Example 객체 -> bytes 변환
    return example.SerializeToString()

In [35]:
serialize_example(False, 4, b'goat', -0.696893709619397)

b'\n:\n\x0b\n\x02f1\x12\x05\x1a\x03\n\x01\x04\n\x0b\n\x02f0\x12\x05\x1a\x03\n\x01\x00\n\x0e\n\x02f2\x12\x08\n\x06\n\x04goat\n\x0e\n\x02f3\x12\x08\x12\x06\n\x04\xa0g2\xbf'

## 출력(저장) 처리

- \_bytes_feature() , \_float_feature() , \_int64_feature() 중 하나를 사용하여 tf.train.Feature로 각각의 값을 변환한 뒤 tf.train.Example 메시지를 만든다.
- serializeToString()을 이용해 binary string 으로 변환한다.
- tf.io.TFRecordWriter를 이용해 출력한다.

In [43]:
# tfreacord 파일을 저장할 디렉토리 생성
import os
os.makedirs('tfrecord', exist_ok=True)

In [44]:
tfrecord_file_path = r'tfrecord/data.tfr'  # TFRecord 파일 확장자: tfr 또는 tfrecord
tfrecord_write = tf.io.TFRecordWriter(tfrecord_file_path) # 저장할 경로를 넣어 생성

for f0, f1, f2, f3 in zip(feature0, feature1, feature2, feature3):
    v = serialize_example(bool(f0), f1, f2, f3) # numpy bool -> 파이썬 bool
    tfrecord_write.write(v) # example 단위로 출력
    
# 연결닫기
tfrecord_write.close()

In [45]:
with tf.io.TFRecordWriter(tfrecord_file_path+'2') as tfrecord_write: # 저장할 경로를 넣어 생성
    for f0, f1, f2, f3 in zip(feature0, feature1, feature2, feature3):
        v = serialize_example(bool(f0), f1, f2, f3) # numpy bool -> 파이썬 bool
        tfrecord_write.write(v) # example 단위로 출력

# 저장된 TFRecord 파일 Loading 및 Dataset 생성
- tfrecord 파일로 저장된 직렬화된 데이터를 읽어 들어와서 feature들을 parsing
- tf.data.TFRecordDataset를 이용해 읽는다.

In [49]:
# 역직렬화(Deserializable): 직렬화된 값(bytes)를 원래 값(객체)로 변환
def deserialize_example(tfrecord_serialized):
    """
    TFRecord에 저장된 직렬화된 한개의 Example(bytes)를 받아서 Feature들을 추출해 input/output으로 나눠 반환하는 함수
    byte -> Example -> Feature -> python타입
    """
    feature_dict = {
        'f0': tf.io.FixedLenFeature([], tf.int64), # f0으로 저장된 Feature를 저장할 빈 Feature
        'f1': tf.io.FixedLenFeature([], tf.int64),
        'f2': tf.io.FixedLenFeature([], tf.string),
        'f3': tf.io.FixedLenFeature([], tf.float32)
    }
    # bytes의 Example을 Example 객체로 역직렬화
    example = tf.io.parse_single_example(tfrecord_serialized, feature_dict)
    
    # Example의 Feature들을 추출해서 python 타입의 값으로 형변환(type casting)
    f0 = tf.cast(example['f0'], tf.bool) # example['f0']: Example에서 'f0'으로 저장된 value(Feature)를 조회
    f1 = tf.cast(example['f1'], tf.int64)
    f2 = tf.cast(example['f2'], tf.string)
    f3 = tf.cast(example['f3'], tf.float32)
    
    return f0, f1, f2, f3

In [50]:
dataset = tf.data.TFRecordDataset(tfrecord_file_path).map(deserialize_example).batch(100)

In [52]:
for batch in dataset.take(5):
    print(np.shape(batch))
    print(batch[0][0], batch[1][0])
    break

(4, 100)
tf.Tensor(False, shape=(), dtype=bool) tf.Tensor(4, shape=(), dtype=int64)


In [46]:
train_dataset = tf.data.TFRecordDataset(tfrecord_file_path)
type(train_dataset)

tensorflow.python.data.ops.readers.TFRecordDatasetV2

In [47]:
for v in train_dataset.take(1):
    print(v)

tf.Tensor(b'\n:\n\x0e\n\x02f3\x12\x08\x12\x06\n\x04\xa0g2\xbf\n\x0b\n\x02f1\x12\x05\x1a\x03\n\x01\x04\n\x0b\n\x02f0\x12\x05\x1a\x03\n\x01\x00\n\x0e\n\x02f2\x12\x08\n\x06\n\x04goat', shape=(), dtype=string)
